<a href="https://colab.research.google.com/github/mazarimono/pycon_mini_hiro/blob/master/pycon_mini_hiro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PyCon mini Hiroshima用ノートブック
#### プレゼン時はズーム200％に

- 家計調査分析
- 利用パッケージ pandas / plotly / dash / jupyter_dash / scikit-learn
- 家計調査のデータを読み込み,前処理し,分析する
- この辺りをもっとやりたい方はPyConJP 2020 Tutorial オープンデータを使って,
Pythonでデータ分析の[リポジトリへ](https://github.com/hannari-python/tutorial)


In [1]:
# 利用パッケージのインストール
!pip install jupyter_dash 
!pip install --upgrade plotly 

     |████████████████████████████████| 51kB 1.5MB/s 
     |████████████████████████████████| 81kB 3.7MB/s 
     |████████████████████████████████| 1.0MB 21.3MB/s 
     |████████████████████████████████| 3.4MB 34.0MB/s 
     |████████████████████████████████| 194kB 40.3MB/s 
     |████████████████████████████████| 1.8MB 31.3MB/s 
     |████████████████████████████████| 358kB 43.0MB/s 
  Created wheel for dash: filename=dash-1.16.2-cp36-none-any.whl size=82323 sha256=50290f0580430b63dcfcb7e4fee31e30c53f13283d334af41ed59f15ccd4929c
  Stored in directory: /root/.cache/pip/wheels/26/de/29/8280c0d15ce1205fd9c6e2819e63ee6125b59dbe75a9a68968
  Created wheel for ansi2html: filename=ansi2html-1.5.2-cp36-none-any.whl size=13862 sha256=f522b970d48561ef71b18173ad4ac4a2808a7a43386e5d5478bc3f4fa3c5a0df
  Stored in directory: /root/.cache/pip/wheels/f9/ea/e0/1ed3c3945ce073f930f3e5aa2b9bbadb03cd2efc8f8695cd61
  Created wheel for flask-compress: filename=Flask_Compress-1.6.0-cp36-none-any.whl size=6454

# データの読み込み
- データの読み込みもPythonを用いると簡単
- 読み込み拒否的なデータもを簡単に読める

In [2]:
# データのダウンロード
# 家計調査・今回利用は小分類・全品目もある（もっと品目が多い）
!wget https://www.stat.go.jp/data/kakei/longtime/csv/h-mon-a.csv



--2020-10-07 00:19:09--  https://www.stat.go.jp/data/kakei/longtime/csv/h-mon-a.csv
Resolving www.stat.go.jp (www.stat.go.jp)... 210.169.215.50, 2001:f6e:500:6b::1
Connecting to www.stat.go.jp (www.stat.go.jp)|210.169.215.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 188 [text/html]
Saving to: ‘h-mon-a.csv’

h-mon-a.csv         100%[===================>]     188  --.-KB/s    in 0s      

2020-10-07 00:19:09 (24.6 MB/s) - ‘h-mon-a.csv’ saved [188/188]



In [3]:
import pandas as pd

In [4]:
# wgetで取得できない
hh = pd.read_csv("/content/h-mon-a.csv")
hh

,<html><head><title>Request Rejected</title></head><body>The requested URL was rejected. Please consult with your administrator.<br><br>Your support ID is: 6133672767911475641</body></html>


In [5]:
import requests
import io 

In [6]:
r= requests.get("https://www.stat.go.jp/data/kakei/longtime/csv/h-mon-a.csv")
r.encoding = "shift-jis"
hh = pd.read_csv(io.StringIO(r.text), encoding="shift-jis")

In [7]:
hh

,二人以上の世帯_支出金額[円],Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,...,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247
0,NaN,NaN,NaN,NaN,NaN,NaN,2000年,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001年,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2002年,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018年,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019年,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020年,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,(平成12年),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(平成13年),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(平成14年),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(平成30年),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(平成31年),NaN,NaN,NaN,(令和元年),NaN,NaN,NaN,NaN,NaN,NaN,NaN,(令和2年),NaN,NaN,NaN,NaN,NaN,NaN
2,表側連番,階層,大分類,中分類,小分類,品目分類,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,11月,12月,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,11月,12月,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,...,4月,5月,6月,7月,8月,9月,10月,11月,12月,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,11月,12月,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,11月,12月,1月,2月,3月,4月,5月,6月,7月
3,1,-,-,-,-,世帯数分布(抽出率調整),10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,...,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
4,2,-,-,-,-,集計世帯数,7887,7942,7934,7922,7928,7917,7907,7908,7917,7937,7914,7933,7888,7887,7907,7927,7948,7924,7907,7909,7910,7908,7914,7924,7857,7877,7915,7894,7932,7926,7914,7893,7914,7933,...,7719,7735,7706,7710,7738,7729,7737,7678,7671,7596,7654,7656,7662,7648,7649,7648,7671,7617,7620,7600,7639,7555,7572,7581,7541,7548,7553,7501,7479,7517,7470,7466,7481,7443,7475,7489,7534,7430,7489,7521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,181,-,-,-,-,会費・つきあい費,1561,1441,1495,1888,1683,1585,1566,1390,1297,1390,1339,1603,1366,1374,1432,1735,1562,1519,1401,1269,1334,1241,1259,1467,1254,1324,1511,1778,1532,1453,1393,1161,1233,1252,...,1007,740,807,751,716,706,713,784,837,737,714,786,948,791,859,788,735,785,762,820,968,832,813,831,1063,878,929,864,707,802,710,776,860,770,705,690,751,696,831,717
184,182,-,-,-,-,その他の教養娯楽,7063,6370,8068,8109,8822,6858,7033,7995,6681,6831,7450,13030,7045,6272,7659,8213,9111,6615,6897,7755,6878,6794,7695,11965,7404,6274,8189,8577,9195,7119,7373,8510,7172,7335,...,9292,9880,8448,8278,9543,8197,7978,8806,12271,8103,7239,9220,8933,9453,7860,8634,9431,8375,8474,8814,11937,8646,7965,8659,9426,10263,8505,8641,9456,9369,8284,9152,12570,8239,7674,8995,8603,9612,8368,8528
185,183,-,-,-,-,消費支出(再掲),309621,290663,335341,335276,308566,297648,326480,309993,296457,309193,297144,391556,308912,293698,339910,318509,300346,286054,314251,302474,283973,305653,294929,359940,305221,273487,330380,320228,291968,289427,315699,298788,295423,304989,...,295929,283056,268802,279197,280320,268802,282872,277361,322157,289703,265614,301230,294439,281307,267641,283387,292481,271273,290396,281041,329271,296345,271232,309274,301136,300901,276882,288026,296327,300609,279671,278765,321380,287173,271735,292214,267922,252017,273699,266897
186,184,-,-,-,-,基礎的支出,184372,170045,186202,173441,177153,169779,180853,179332,166901,174690,170795,228730,188487,168993,184419,172758,174311,163180,175597,178844,164006,171481,170254,218506,184845,164207,18

# CSVファイルを取得してもそのままは使えない
- データを取得しても使いやすい形に整形する必要がある
- pandasを用いると容易に加工できる
- というわけで以下ではその処理を行ってみます

In [8]:
# 日付の作成
## 開始年月　- 終了年月の作成

year_list = hh.iloc[0,:].dropna().values # 年のデータの取得
month_list = hh.iloc[2, 6:].dropna().values # 月のデータの取得

year_list

array(['2000年', '2001年', '2002年', '2003年', '2004年', '2005年', '2006年',
       '2007年', '2008年', '2009年', '2010年', '2011年', '2012年', '2013年',
       '2014年', '2015年', '2016年', '2017年', '2018年', '2019年', '2020年'],
      dtype=object)

In [9]:
# 処理に用いれるよう、数値のリストに変換
year_list_replace = [int(year.replace("年", "")) for year in year_list]
month_list_replace  = [int(month.replace("月", "")) for month in month_list]

In [10]:
# date_range関数を使った日付の作成
hh_date_range = pd.date_range(f"{year_list_replace[0]}-{month_list_replace[0]}", f"{year_list_replace[-1]}-{month_list_replace[-1]+1}", freq="M")

In [11]:
hh_data = hh.iloc[3:, 5:].copy()
hh_data = hh_data.set_index("Unnamed: 5")
hh_data.columns = hh_date_range
hh_data.index.name = "" 
data = hh_data.T.copy()
data.index = pd.to_datetime(data.index)
data = data.astype("float")

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 247 entries, 2000-01-31 to 2020-07-31
Freq: M
Columns: 185 entries, 世帯数分布(抽出率調整) to 選択的支出
dtypes: float64(185)
memory usage: 358.9 KB


In [13]:
data.head()

,世帯数分布(抽出率調整),集計世帯数,世帯人員(人),18歳未満人員(人),65歳以上人員(人),うち無職者人員(人),有業人員(人),世帯主の年齢(歳),持家率(％),家賃・地代を支払っている世帯の割合(％),消費支出,食料,穀類,米,パン,麺類,他の穀類,魚介類,生鮮魚介,塩干魚介,魚肉練製品,他の魚介加工品,肉類,生鮮肉,加工肉,乳卵類,牛乳,乳製品,卵,野菜・海藻,生鮮野菜,乾物・海藻,大豆加工品,他の野菜・海藻加工品,果物,生鮮果物,果物加工品,油脂・調味料,油脂,調味料,...,被服及び履物(財・サービス),半耐久財(被服及び履物),サービス(被服及び履物),保健医療(財・サービス),耐久財(保健医療),半耐久財(保健医療),非耐久財(保健医療),サービス(保健医療),交通・通信(財・サービス),耐久財(交通・通信),半耐久財(交通・通信),非耐久財(交通・通信),サービス(交通・通信),教育(財・サービス),非耐久財(教育),サービス(教育),教養娯楽(財・サービス),耐久財(教養娯楽),半耐久財(教養娯楽),非耐久財(教養娯楽),サービス(教養娯楽),諸雑費(財・サービス),耐久財(諸雑費),半耐久財(諸雑費),非耐久財(諸雑費),サービス(諸雑費),調整集計世帯数,教養娯楽関係費,耐久財(教養娯楽関係費),他の教養娯楽関係費,読書,聴視・観覧,旅行,スポーツ,月謝,会費・つきあい費,その他の教養娯楽,消費支出(再掲),基礎的支出,選択的支出
2000-01-31,10000.0,7887.0,3.32,0.74,0.52,0.41,1.51,52.4,76.0,22.1,309621.0,73580.0,6100.0,2338.0,2127.0,1325.0,310.0,8525.0,5479.0,1370.0,819.0,858.0,6229.0,5144.0,1085.0,3390.0,1590.0,1114.0,686.0,8029.0,4702.0,741.0,1250.0,1336.0,2866.0,2660.0,205.0,2897.0,264.0,2633.0,...,19602.0,18712.0,891.0,10941.0,1123.0,422.0,3410.0,5986.0,31386.0,4435.0,1025.0,3876.0,22049.0,12532.0,281.0,12251.0,31681.0,4163.0,4624.0,6818.0,16075.0,19157.0,797.0,2064.0,4047.0,12249.0,736445.0,34008.0,3912.0,30096.0,4304.0,2049.0,8876.0,3001.0,3241.0,1561.0,7063.0,309621.0,184372.0,125248.0
2000-02-29,10000.0,7942.0,3.32,0.75,0.53,0.41,1.51,52.6,76.3,22.0,290663.0,73309.0,6915.0,2919.0,2329.0,1359.0,308.0,8627.0,5475.0,1500.0,802.0,850.0,6449.0,5242.0,1208.0,3650.0,1678.0,1200.0,772.0,8570.0,5102.0,744.0,1298.0,1427.0,3168.0,3001.0,167.0,3111.0,272.0,2838.0,...,14400.0,13571.0,830.0,11789.0,853.0,443.0,3597.0,6896.0,31041.0,5455.0,887.0,3856.0,20843.0,14481.0,220.0,14260.0,29737.0,3026.0,3937.0,6892.0,15882.0,18800.0,731.0,2473.0,4291.0,11306.0,735391.0,32103.0,2920.0,29183.0,4586.0,2199.0,9058.0,2200.0,3329.0,1441.0,6370.0,290663.0,170045.0,120618.0
2000-03-31,10000.0,7934.0,3.32,0.75,0.53,0.41,1.51,52.7,76.2,22.6,335341.0,79726.0,7496.0,3226.0,2492.0,1482.0,296.0,9268.0,5880.0,1613.0,794.0,981.0,6483.0,5170.0,1314.0,3940.0,1835.0,1300.0,804.0,9268.0,5630.0,841.0,1311.0,1487.0,3270.0,3111.0,159.0,3262.0,308.0,2954.0,...,18366.0,17174.0,1192.0,11767.0,1043.0,368.0,3891.0,6464.0,39097.0,9004.0,1542.0,4206.0,24345.0,17709.0,1100.0,16609.0,36010.0,4235.0,5050.0,8355.0,18370.0,22685.0,1006.0,2816.0,4896.0,13968.0,735404.0,39415.0,4248.0,35166.0,4858.0,1977.0,12249.0,3022.0,3497.0,1495.0,8068.0,335341.0,186202.0,149139.0
2000-04-30,10000.0,7922.0,3.32,0.75,0.52,0.41,1.52,52.6,75.8,22.2,335276.0,77344.0,7470.0,3346.0,2441.0,1458.0,225.0,8736.0,5438.0,1571.0,761.0,966.0,6485.0,5082.0,1404.0,3803.0,1769.0,1264.0,770.0,9268.0,5762.0,828.0,1262.0,1415.0,2999.0,2848.0,152.0,3149.0,315.0,2834.0,...,18028.0,16141.0,1887.0,11251.0,1000.0,286.0,3340.0,6624.0,41356.0,10981.0,1563.0,4087.0,24725.0,24049.0,926.0,23124.0,34146.0,2890.0,4777.0,8519.0,17960.0,20400.0,580.0,2216.0,4528.0,13076.0,734606.0,37458.0,3109.0,34349.0,4552.0,2470.0,10432.0,3129.0,3768.0,1888.0,8109.0,335276.0,173441.0,161835.0
2000-05-31,10000.0,7928.0,3.31,0.75,0.52,0.41,1.53,52.7,76.1,22.6,308566.0,81415.0,7447.0,3264.0,2458.0,1523.0,203.0,8857.0,5554.0,1586.0,756.0,961.0,6655.0,5168.0,1487.0,3899.0,1877.0,1258.0,763.0,9533.0,5983.0,767.0,1314.0,1469.0,3148.0,2985.0,162.0,3307.0,321.0,2985.0,...,18712.0,16753.0,1960.0,11335.0,965.0,513.0,3370.0,6487.0,36089.0,6613.0,1158.0,4467.0,23852.0,11512.0,427.0,11084.0,33824.0,2517.0,3666.0,9032.0,18608.0,19449.0,452.0,2321.0,4698.0,11979.0,733789.0,38525.0,3354.0,35171.0,4516.0,2202.0,11746.0,2946.0,3256.0,1683.0,8822.0,308566.0,177153.0,131413.0


# データが整形できたので次にデータの傾向などを観察します
- plotlyを使ってデータの観察

In [14]:
import plotly.express as px

# 先ほど作成したデータをそのまま可視化

In [20]:
px.line(data, x=data.index, y=["世帯人員(人)"])

# 一人当たりの消費金額のデータを作成

In [21]:
data_each = data.apply(lambda x: x/x[2], axis=1)
data_each = data_each.iloc[:, 10:]

In [24]:
px.line(data_each, x=data_each.index, y=["乳製品"])

In [26]:
# 各年度の比較を視覚化したい
data["month"] = data.index.map(lambda x: x.month)
data["year"] = data.index.map(lambda x: x.year)
data_each["month"] = data_each.index.map(lambda x: x.month)
data_each["year"] = data_each.index.map(lambda x: x.year)

In [27]:
px.line(data_each, x="month", y="消費支出", color="year")

# いちいち可視化するのに切り替えるの面倒
- そんなあなたにdash 
- dashをjupyterで使うパッケージJupyterDashを使おう！

In [28]:
from jupyter_dash import JupyterDash 
import dash_core_components as dcc 
import dash_html_components as html 
from dash.dependencies import Input, Output, State, MATCH

In [29]:
app = JupyterDash(__name__)

app.layout = html.Div([
                       dcc.RadioItems(id="radio",
                                      options=[{"label": i, "value": i} for i in ["世帯", "1人当たり"]],
                                      value="世帯"
                                      ),
                       dcc.Dropdown(id="drop", multi=True), # 中身はラジオボタンの選択により変わる
                      dcc.Graph(id="graph")
])

@app.callback(Output("drop", "options"), Output("drop", "value"), Input("radio", "value"))
def update_dropdown(radio_value):
  if radio_value == "世帯":
    drop_options = [{"value": i, "label": i} for i in data.columns]
    first_value = data.columns[2]
    return drop_options, first_value 
  else:
    drop_options = [{"value": i, "label": i} for i in data_each.columns]
    first_value = data_each.columns[0]
    return drop_options, first_value

@app.callback(Output("graph", "figure"), Input("radio", "value"), Input("drop", "value"))
def update_graph(radio_value, selected_values):
  if len(selected_values) == 0:
    raise dash.exceptions.PreventUpdate
  if radio_value == "世帯":
    return px.line(data, x=data.index, y=selected_values)
  else:
    return px.line(data_each, x=data_each.index, y=selected_values)

app.run_server(mode="inline")

<IPython.core.display.Javascript object>

In [30]:
from sklearn.cluster import KMeans

In [31]:
# 直近1年分のデータをクラスタリング
# 用のデータの作成(指数化)
data_each_drop = data_each.drop(["month", "year"], axis=1)
data_each_drop_year = data_each_drop["2020-1":].copy()
data_each_drop_year_index = data_each_drop_year / data_each_drop_year.iloc[0, :] * 100


In [32]:
data_trans = data_each_drop_year_index.T

In [34]:
model = KMeans(n_clusters=5)
label = model.fit_predict(data_trans)

In [35]:
data_trans["label"] = label
df_table = pd.DataFrame(data_trans["label"].value_counts()).reset_index()

In [40]:
df_table

,index,label
0,2,101
1,3,50
2,0,21
3,1,2
4,4,1


In [36]:
model.cluster_centers_[0, :]

array([100.        , 124.33943584, 152.26031277, 134.79533344,
       140.33786911, 169.63722528, 164.22123947])

In [43]:
len(model.cluster_centers_)

5

In [44]:
import plotly.graph_objects as go

In [46]:
fig = go.Figure()

for i in range(len(model.cluster_centers_)):
  fig.add_trace(go.Scatter(x=data_trans.columns[:-1], y=model.cluster_centers_[i, :], name=i))

fig.show()

In [51]:
data_trans[data_trans["label"]==1].index

Index(['教科書・学習参考教材', '非耐久財(教育)'], dtype='object', name='')

In [52]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
                       html.Div([
                       html.H1("家計調査クラスタリング"),
                       dcc.Input(id="cluster_num", type="number", value=5, style={"textAlign":"center"}),
                       ], style={"textAlign": "center"}),
                       html.Div([
                                 html.Div([
                                           dcc.Graph(id="graph1")
                                 ], style={"width": "60%", "display": "inline-block"}),
                                 html.Div([
                                           dcc.Graph(id="graph2"),
                                 ], style={"width": "40%", "display": "inline-block"})

                       ]),

], style={"margin": "2%"})

@app.callback(Output("graph1", "figure"), Output("graph2", "figure"),Input("cluster_num", "value"))
def update_everything(cluster_num):
  model = KMeans(n_clusters=cluster_num)
  label = model.fit_predict(data_trans)
  data_trans["label"] = label 
  df_label = pd.DataFrame(data_trans["label"].value_counts()).reset_index() # ラベル数の集計のデータフレーム
  fig = go.Figure()
  for i in range(len(model.cluster_centers_)):
    fig.add_trace(go.Scatter(x=data_trans.columns[:-1], y=model.cluster_centers_[i, :], name=i))
  table = go.Figure(data=go.Table(header={"values": df_label.columns}, cells={"values": [df_label[col].tolist() for col in df_label.columns]}))
  return fig, table


app.run_server(mode="inline")

<IPython.core.display.Javascript object>

## Pythonとオープンデータ
- Pythonでデータ収集、前処理、分析を行った
- オープンデータを触ることにより自社が必要であるデータが分かる
- 

# オープンデータ　種類も豊富
- 家計調査に絡むデータも豊富
- 家計調査はもっと詳細なものがあるとともに、生の調査データを買えるサービスも
- 実データは出されていないものの、1週前までのPOSデータが使われたデータも

In [ ]:
# 家計調査ミクロデータ（1件ずつの調査データが入っている）
!wget https://github.com/hannari-python/tutorial/raw/master/data/ippan-microdata/ippan_2009zensho_s.zip
!unzip ippan_2009zensho_s.zip

# METI POS小売販売額指標（https://www.meti.go.jp/statistics/bigdata-statistics/bigdata_pj_2019/pos_gfk_intage.html）
!wget https://raw.githubusercontent.com/mazarimono/pycon_mini_hiro/master/data/meti_pos_data.csv

--2020-10-06 07:56:21--  https://github.com/hannari-python/tutorial/raw/master/data/ippan-microdata/ippan_2009zensho_s.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hannari-python/tutorial/master/data/ippan-microdata/ippan_2009zensho_s.zip [following]
--2020-10-06 07:56:22--  https://raw.githubusercontent.com/hannari-python/tutorial/master/data/ippan-microdata/ippan_2009zensho_s.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33090889 (32M) [application/zip]
Saving to: ‘ippan_2009zensho_s.zip.1’

ippan_2009zensho_s. 100%[===================>]  31.56M  58.8MB/s    in 0.5s    

2020-10-06 07:

In [ ]:
micro = pd.read_csv("/content/ippan_2009zensho_s/ippan_2009zensho_s_dataset.csv", encoding="shift-jis", header=8)
micro

,3City,T_SeJinin,T_SyuJinin,T_JuSyoyu,T_Syuhi,T_Age_5s,T_Age_65,Weight,Y_Income,L_Expenditure,Food,Housing,LFW,Furniture,Clothes,Health,Transport,Education,Recreation,OL_Expenditure,E001,E002,E003,E004,E005,E006,E007,E008,E009,E010,E011,E012,E013,E014,E015,E016,E017,E018,E019,E020,...,E371,E372,E373,E374,E375,E376,E377,E378,E379,E380,E381,E382,E383,E384,E385,E386,E387,E388,E389,E390,E391,E392,E393,E394,E395,E396,E397,E398,E399,E400,E401,E402,E403,E404,E405,E406,E407,E408,E409,E410
0,1,2,1,1,1,1,1,895.266667,3917,201649,47756,16028,9652,6702,8088,726,21546,0,14433,76719,47756,4574,1728,1557,1027,262,3450,2070,555,393,432,3722,2882,785,1286,621,64,126,840,636,...,1320,2186,6448,138,512,535,582,277,4404,2885,65,939,1089,224,538,31,2240,18347,1185,57,5425,244,284,9490,231,95,182,1154,17820,15248,2572,24149,19000,5149,417,3591,1141,576,186,390
1,1,2,1,1,1,1,1,895.266667,6675,166381,34054,7416,26313,17062,6989,7637,20773,0,19048,27089,34054,2813,1019,1050,489,254,1863,905,564,132,261,2432,2112,632,621,697,23,139,320,203,...,231,668,1677,21,258,100,78,38,1182,1581,26,899,189,161,291,14,656,5794,229,10,3980,49,96,804,96,32,95,404,7335,6811,523,8710,7687,1023,239,437,348,282,94,188
2,1,2,1,1,1,1,1,895.266667,6706,259736,84501,1927,10082,6741,5090,11015,53372,0,17289,69719,84501,5004,657,1122,3067,158,7010,4216,718,836,1239,7742,5284,3382,1136,628,43,95,2458,2248,...,4942,3637,8037,190,1210,1606,423,557,4050,8032,124,3204,3350,316,1007,30,1096,20994,1447,40,4398,497,258,9418,295,75,121,4445,9115,8095,1021,11933,7405,4529,1166,2705,658,429,183,246
3,1,2,1,1,1,1,1,895.266667,2790,114511,41664,730,22358,5413,1205,5049,17411,0,8605,12077,41664,4372,1785,1474,910,202,3680,2713,363,255,349,3093,2512,925,828,565,45,149,581,383,...,306,113,2073,36,116,202,109,56,1554,487,18,116,147,45,146,14,182,1661,45,3,157,349,63,667,34,6,24,314,2692,2291,401,4136,2644,1492,69,977,445,205,55,151
4,1,2,1,1,1,1,1,895.266667,2577,193505,56981,3779,28747,4812,4243,751,16435,0,38231,39527,56981,4957,1463,1463,1763,268,4712,3181,558,413,560,4644,3726,1310,1682,518,87,129,919,637,...,842,699,7654,46,160,131,612,93,6613,1780,7,643,870,170,82,8,1116,4768,280,22,2000,164,50,1347,17,96,133,660,5131,3859,1272,16907,4840,12068,699,10940,428,445,138,307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45806,0,3,2,2,2,0,2,552.266667,4398,216774,46169,39809,32508,2896,4203,18778,21739,30,4315,46326,46169,5207,2668,1391,913,235,4929,3086,810,515,518,3851,3055,1001,1263,607,64,119,797,604,...,1022,1163,3508,82,243,280,362,165,2376,1196,40,576,276,67,193,45,1122,12331,1560,291,728,1149,292,6481,243,48,904,634,10563,7368,3194,12858,10883,1975,175,1076,724,1817,957,860
45807,0,3,2,2,2,0,2,552.266667,4844,165978,52670,37839,25403,3532,2293,7016,5022,8,12656,19540,52670,5932,3008,1602,1051,271,5653,3556,927,578,593,4410,3527,1164,1477,680,71,134,883,669,...,408,335,1003,43,200,128,150,76,406,527,19,231,172,20,68,16,516,3927,727,82,125,888,81,1493,115,18,191,208,6634,5082,1552,5104,4120,984,89,523,372,769,363,406
45808,0,3,2,2,2,0,2,552.266667,4630,244064,53784,76871,18196,17747,3308,13499,27705,25,4309,28619,53784,6033,3086,1614,1052,280,5761,3633,932,596,599,4503,3581,1184,1485,700,75,138,922,707,...,626,757,2547,41,121,227,423,104,1630,936,26,282,295,38,266,29,402,6524,847,117,300,812,186,3022,187,38,608,408,6198,3433,2765,8804,7553,1251,190,699,362,1431,973,457
45809,0,3,2,2,2,0,2,552.266667,6738,399003,123080,38810,19711,2816,5666,21831,23914,28,58844,104302,123080,13423,6772,3770,2247,634,13315,8362,2196,1353,1405,10379,8271,2713,3424,1644,174,316,2108,1603,...,2179,2480,7502,198,515,597,772,353,5066,2641,85,1228,606,119,508,95,2025,26762,3328,817,1364,3160,624,13367,517,102,1928,1554,27466,16321,11145

In [ ]:
micro = pd.read_csv("/content/meti_pos_data.csv")
micro

,期間_表示,週次・月次(フィルタ用),地域名,業態名,大分類名,中分類名,中分類名 (経産省_ダッシュボード_POS前年比_公表コントロール.csv),地域名 (経産省_ダッシュボード_POS前年比_公表コントロール.csv),大分類名 (経産省_ダッシュボード_POS前年比_公表コントロール.csv),月次・週次,期間,期間_最大（フィルタ用）,期間_表示（年）,期間_表示（最新時点）,期間_表示（月・ISO週番号）,業態名 (経産省_ダッシュボード_POS前年比_公表コントロール.csv),非公開フラグ,非公開フラグ (経産省_ダッシュボード_POS前年比_公表コントロール.csv),計算1_前年比,販売金額（原指数）,金額前年比
0,2014-08-17,週次,02_東北,03_ホームセンター,40_化粧品,00_中分類計,NaN,NaN,NaN,W,2014年8月11日,偽,2014,2014-08-11～2014-08-17 [33],33,NaN,0,NaN,NaN,103.96,-20.36
1,2014-08-24,週次,02_東北,03_ホームセンター,40_化粧品,00_中分類計,NaN,NaN,NaN,W,2014年8月18日,偽,2014,2014-08-18～2014-08-24 [34],34,NaN,0,NaN,NaN,89.35,-7.78
2,2014-08-31,週次,02_東北,03_ホームセンター,40_化粧品,00_中分類計,NaN,NaN,NaN,W,2014年8月25日,偽,2014,2014-08-25～2014-08-31 [35],35,NaN,0,NaN,NaN,98.13,-4.85
3,2014-09-07,週次,02_東北,03_ホームセンター,40_化粧品,00_中分類計,NaN,NaN,NaN,W,2014年9月1日,偽,2014,2014-09-01～2014-09-07 [36],36,NaN,0,NaN,NaN,88.39,-4.67
4,2014-09-14,週次,02_東北,03_ホームセンター,40_化粧品,00_中分類計,NaN,NaN,NaN,W,2014年9月8日,偽,2014,2014-09-08～2014-09-14 [37],37,NaN,0,NaN,NaN,98.15,-2.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175001,2020-08-23,週次,15_中国・四国・九州・沖縄,05_家電大型専門店,70_家電,75_エアコン,NaN,NaN,NaN,W,2020年8月17日,偽,2020,2020-08-17～2020-08-23 [34],34,NaN,0,NaN,NaN,249.20,98.59
175002,2020-08-30,週次,15_中国・四国・九州・沖縄,05_家電大型専門店,70_家電,75_エアコン,NaN,NaN,NaN,W,2020年8月24日,偽,2020,2020-08-24～2020-08-30 [35],35,NaN,0,NaN,NaN,151.85,35.90
175003,2020-09-06,週次,15_中国・四国・九州・沖縄,05_家電大型専門店,70_家電,75_エアコン,NaN,NaN,NaN,W,2020年8月31日,偽,2020,2020-08-31～2020-09-06 [36],36,NaN,0,NaN,NaN,118.30,-27.09
175004,2020-09-13,週次,15_中国・四国・九州・沖縄,05_家電大型専門店,70_家電,75_エアコン,NaN,NaN,NaN,W,2020年9月7日,偽,2020,2020-09-07～2020-09-13 [37],37,NaN,0,NaN,NaN,88.78,-49.63
